In [1]:
import pandas as pd
import scipy as sp
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from math import log
import re

In [2]:
df_train = pd.read_json('/Users/sroy/Data/Two_Sigma/train.json')
df_test= pd.read_json('/Users/sroy/Data/Two_Sigma/test.json')

In [3]:
df_train.columns

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address'],
      dtype='object')

In [4]:
df_train.shape

(49352, 15)

In [5]:
df_train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [6]:
def percentile_df(df,col_name,llimit,ulimit):
    df_model = df[(df[col_name] < np.percentile(df[col_name],ulimit))& (df[col_name] > np.percentile(df[col_name],llimit))]
    return df_model

In [7]:
df_model = percentile_df(df_train,'latitude',10,90)

In [8]:
df_model =  percentile_df(df_train,'longitude',10,90)

In [9]:
df_model = percentile_df(df_train,'price',0,90)

In [10]:
df_model['desc_feature'] = df_model['description'].apply(lambda x: 0 if len(x.strip())==0 else 1 )

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
df_model['display_address_feature'] = df_model['display_address'].apply(lambda x: 0 if len(x.strip())==0 else 1 )

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
df_model['price_feature'] = df_model['price'].apply(lambda x: log(x))

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df_model['photos_feature'] = df_model['photos'].apply(lambda x : len(x))

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
df_model['fee_feature'] = df_model['features'].apply(lambda x:  0 
                                                 if re.match('(.*)fee(.*)',''.join(x),re.IGNORECASE) is None else 1 )

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
df_model['number_feature'] = df_model['features'].apply(lambda x: len(x))

/Users/sroy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
df_model.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,...,manager_id,photos,price,street_address,desc_feature,display_address_feature,price_feature,photos_feature,fee_feature,number_feature
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,...,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,1,1,8.006368,5,0,0
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,...,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,0,1,8.606119,11,0,5
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,...,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,1,1,7.955074,8,0,4
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,...,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,1,1,8.094073,3,1,2
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,...,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,1,1,8.116716,3,0,1


In [17]:
# df_model_train= df_model[['interest_level','bathrooms','bedrooms','latitude','longitude','desc_feature','display_address_feature','price_feature',
#                     'photos_feature','fee_feature','number_feature']]

df_model_train= df_model[['interest_level','bathrooms','bedrooms','desc_feature','display_address_feature','price_feature',
                    'photos_feature','fee_feature','number_feature']]

In [18]:
df_model_train.head()

,interest_level,bathrooms,bedrooms,desc_feature,display_address_feature,price_feature,photos_feature,fee_feature,number_feature
10,medium,1.5,3,1,1,8.006368,5,0,0
10000,low,1.0,2,0,1,8.606119,11,0,5
100004,high,1.0,1,1,1,7.955074,8,0,4
100007,low,1.0,1,1,1,8.094073,3,1,2
100013,low,1.0,4,1,1,8.116716,3,0,1


In [19]:
# X_train = df_model_train[['bathrooms','bedrooms','latitude','longitude','desc_feature','display_address_feature','price_feature',
#                     'photos_feature','fee_feature','number_feature']]

X_train = df_model_train[['bathrooms','bedrooms','desc_feature','display_address_feature','price_feature',
                    'photos_feature','fee_feature','number_feature']]
Y_train = df_model_train[['interest_level']]

In [20]:
clf = LinearDiscriminantAnalysis(solver="svd")

In [21]:
clf.fit(X_train, Y_train)

/Users/sroy/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [22]:
df_test['desc_feature'] = df_test['description'].apply(lambda x: 0 if len(x.strip())==0 else 1 )
df_test['display_address_feature'] = df_test['display_address'].apply(lambda x: 0 if len(x.strip())==0 else 1 )
df_test['price_feature'] = df_test['price'].apply(lambda x: log(x))
df_test['photos_feature'] = df_test['photos'].apply(lambda x : len(x))
df_test['fee_feature'] = df_test['features'].apply(lambda x:  0 
                                                 if re.match('(.*)fee(.*)',''.join(x),re.IGNORECASE) is None else 1 )
df_test['number_feature'] = df_test['features'].apply(lambda x: len(x))

In [23]:
# X_test = df_test[['bathrooms','bedrooms','latitude','longitude','desc_feature','display_address_feature','price_feature',
#                     'photos_feature','fee_feature','number_feature']]

X_test = df_test[['bathrooms','bedrooms','desc_feature','display_address_feature','price_feature',
                    'photos_feature','fee_feature','number_feature']]

In [24]:
pred =clf.predict(X_test)
prob =clf.predict_proba(X_test)
df_output=pd.DataFrame(prob)
df_output.to_csv('lda_submission.csv')
clf.classes_

array(['high', 'low', 'medium'], 
      dtype='<U6')

In [25]:
df_output.head()

,0,1,2
0,0.054789,0.696217,0.248994
1,0.091834,0.620930,0.287236
2,0.039171,0.718421,0.242409
3,0.066713,0.631500,0.301787
4,0.044076,0.636178,0.319746
